In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요한 모듈 import
import pandas as pd
import ast
from collections import Counter

본 팀에서 크롤링한 무신사 태그 데이터에는 <무신사에서 판매하는 패션 제품 카테고리>, <코디 스타일> 정보가 각 코디마다 동일한 형식으로 포함되어있음. 이는 의미 있는 태그를 추출하기 위한 과정에서 필요없는(방해가 되는) 태그라고 판단하여 분석을 위한 데이터에서 제거하기로 함.

이는 위의 내용을 전처리하기 위한 코드 파일임.

In [ ]:
# 무신사 내 패션 제품 카테고리를 리스트 형태로 저장
product_list = [
    '후드 티셔츠','맨투맨/스웨트셔츠','니트/스웨터','셔츠/블라우스','긴소매 티셔츠','반소매 티셔츠','민소매 티셔츠','피케/카라 티셔츠','기타 상의',
    '후드 집업','블루종/MA-1','레더/라이더스 재킷','무스탕/퍼','트러커 재킷','슈트/블레이저 재킷','카디건','아노락 재킷','플리스/뽀글이','트레이닝 재킷','스타디움 재킷',
    '환절기 코트','겨울 싱글 코트','겨울 더블 코트','겨울 기타 코트','롱패딩/롱헤비 아우터','숏패딩/숏헤비 아우터','패딩 베스트','베스트','사파리/헌팅 재킷','나일론/코치 재킷','기타 아우터',
    '데님 팬츠','코튼 팬츠','슈트 팬츠/슬랙스','트레이닝/조거 팬츠','숏 팬츠','레깅스','점프 슈트/오버올','기타 바지',
    '미니 원피스','미디 원피스','맥시 원피스','미니스커트','미디스커트','롱스커트',
    '구두','로퍼','힐/펌프스','플랫 슈즈','블로퍼','샌들','슬리퍼','기타 신발','모카신/보트 슈즈','부츠','캔버스/단화','패션스니커즈화','기타 스니커즈',
    '백팩','메신저/크로스 백','숄더백','토트백','에코백','보스턴/드럼/더플백','웨이스트 백','파우치 백','브리프케이스','캐리어','가방 소품','지갑/머니클립','클러치 백','크로스백',
    '캡/야구 모자','헌팅캡/베레모','페도라','버킷/사파리햇','비니','트루퍼','기타 모자',
    '디지털','쿼츠 아날로그','오토매틱 아날로그','시계 용품','기타 시계',
    '팔찌','반지','목걸이/펜던트','귀걸이','발찌','브로치/배지','헤어 액세서리',
    '안경','선글라스','안경 소품','양말','스타킹','마스크','키링/키케이스','벨트','넥타이','머플러','스카프/반다나','장갑','기타 액세서리',
    '여성 속옷 상의','여성 속옷 하의','여성 속옷 세트','남성 속옷','홈웨어','수영복/비치웨어'
]


# 무신사 내 패션 코디 스타일 태그를 리스트 형태로 저장
style_list = [
    '아메리칸 캐주얼','캐주얼','시크','댄디','포멀','걸리시','골프','레트로','로맨틱','스포츠','스트릿'
]

In [ ]:
# 남자 Database csv 파일 불러오기
man = pd.read_csv('/content/drive/MyDrive/2023 KUIAI 해커톤/Data/Man/Man.csv', encoding = 'cp949', index_col = 0)
man

,image_id,title,tags_list,name,gender,style
0,31162,스트릿 패션,"['#겨울', '#스트릿', '#컬러포인트', '#글로시', '# 카고', '# 와...",김성주,1,#스트릿
1,31161,아메카지 룩,"['#겨울', '#아메리칸 캐주얼', '# 퀼팅', '# 와이드핏', '#핀턱', ...",김성주,1,#아메리칸 캐주얼
2,31160,유니크한 조합,"['#겨울', '#스트릿', '#컬러포인트', '#레터링', '# 코듀로이', '#...",김성주,1,#스트릿
3,31159,캐주얼 무드,"['#겨울', '#캐주얼', '#얼죽코', '# 싱글코트', '#컬러블록', '# ...",김성주,1,#캐주얼
4,31158,데일리하게,"['#겨울', '#캐주얼', '#스티치', '# 그래픽', '#언밸런스', '# 와...",김성주,1,#캐주얼
...,...,...,...,...,...,...
2061,26762,스포티 무드,"['#컬러포인트', '#컬러블록', '#레터링', '#로고플레이', '#스포티', ...",윤지욱,1,#골프
2062,26761,캠핑 룩으로 제격,"['#페이즐리', '#오버핏', '#캠핑', '#봄', '#아메리칸 캐주얼', '#...",윤지욱,1,#아메리칸 캐주얼
2063,26760,깔끔한 스타일,"['#스트라이프', '#컬러블록', '#꾸안꾸', '#봄', '#캐주얼', '#그라...",윤지욱,1,#캐주얼
2064,26759,따라 입어 봐!,"['#펀칭', '#오버핏', '#크롭', '#톤온톤', '#뉴트럴톤', '#봄', ...",윤지욱,1,#캐주얼


In [ ]:
# 인코딩 과정의 오류를 해결하기 위한 코드
'''
csv 파일의 'tags_list' column의 데이터 타입이 인코딩 과정에서 리스트가 아닌 문자열로 저장이 되었음.
이를 다시 리스트 타입으로 변환하기 위한 코드 블록임.
'''
for i in range(len(man)):
  tag_list = man['tags_list'].iloc[i]
  tag_list = ast.literal_eval(tag_list)
  man['tags_list'].iloc[i] = tag_list

In [ ]:
man
# 정상적으로 'tags_list' column이 list type으로 변환되었음.

,image_id,title,tags_list,name,gender,style
0,31162,스트릿 패션,"[#겨울, #스트릿, #컬러포인트, #글로시, # 카고, # 와이드핏, #컬러블록,...",김성주,1,#스트릿
1,31161,아메카지 룩,"[#겨울, #아메리칸 캐주얼, # 퀼팅, # 와이드핏, #핀턱, #컬러포인트, # ...",김성주,1,#아메리칸 캐주얼
2,31160,유니크한 조합,"[#겨울, #스트릿, #컬러포인트, #레터링, # 코듀로이, # 플라워, #페이즐리...",김성주,1,#스트릿
3,31159,캐주얼 무드,"[#겨울, #캐주얼, #얼죽코, # 싱글코트, #컬러블록, # 워싱, # 와이드핏,...",김성주,1,#캐주얼
4,31158,데일리하게,"[#겨울, #캐주얼, #스티치, # 그래픽, #언밸런스, # 와이드핏, #니트/스웨...",김성주,1,#캐주얼
...,...,...,...,...,...,...
2061,26762,스포티 무드,"[#컬러포인트, #컬러블록, #레터링, #로고플레이, #스포티, #레저스포츠, #슬...",윤지욱,1,#골프
2062,26761,캠핑 룩으로 제격,"[#페이즐리, #오버핏, #캠핑, #봄, #아메리칸 캐주얼, #그라미치, #츄바스코...",윤지욱,1,#아메리칸 캐주얼
2063,26760,깔끔한 스타일,"[#스트라이프, #컬러블록, #꾸안꾸, #봄, #캐주얼, #그라미치, #리복 워치,...",윤지욱,1,#캐주얼
2064,26759,따라 입어 봐!,"[#펀칭, #오버핏, #크롭, #톤온톤, #뉴트럴톤, #봄, #캐주얼, #반스, #...",윤지욱,1,#캐주얼


In [ ]:
# 모든 이미지가 가지고 있는 태그들을 합쳐서 하나의 태그 리스트로 만든 코드임.
uniq_tags_list = list(man['tags_list'].sum())
print(uniq_tags_list)
print(len(uniq_tags_list))

['#겨울', '#스트릿', '#컬러포인트', '#글로시', '# 카고', '# 와이드핏', '#컬러블록', '# 퀼팅', '#유니크', '#긴소매 티셔츠', '#데님 팬츠', '#디앤에스알', '#베스트', '#선글라스', '#숏패딩/숏헤비 아우터', '#에러스익셉티드', '#유니폼브릿지', '#이티씨이', '#장갑', '#척', '#캡/야구 모자', '#투큐브', '#패션스니커즈화', '#프로스펙스', '#피스메이커', '#겨울', '#아메리칸 캐주얼', '# 퀼팅', '# 와이드핏', '#핀턱', '#컬러포인트', '# 아메카지', '#구두', '#기타 바지', '#기타 아우터', '#레테르코모', '#머플러', '#메신저/크로스 백', '#모카신/보트 슈즈', '#밀로', '#선데이후르츠마켓', '#에러스익셉티드', '#이티씨이', '#클래티지', '#헌팅캡/베레모', '#후드 티셔츠', '#겨울', '#스트릿', '#컬러포인트', '#레터링', '# 코듀로이', '# 플라워', '#페이즐리', '#유니크', '#그레고리', '#긴소매 티셔츠', '#레테르코모', '#메신저/크로스 백', '#목걸이/펜던트', '#버킷/사파리햇', '#선글라스', '#선데이후르츠마켓', '#슬리퍼', '#아웃 오브 트렁크', '#오와이', '#오찌', '#투큐브', '#트레이닝/조거 팬츠', '#패딩 베스트', '#포콤마', '#겨울', '#캐주얼', '#얼죽코', '# 싱글코트', '#컬러블록', '# 워싱', '# 와이드핏', '# 그래픽', '#얼죽코', '# 싱글코트', '#컬러블록', '# 워싱', '# 와이드핏', '# 그래픽', '#86로드', '#겨울 기타 코트', '#데님 팬츠', '#메신저/크로스 백', '#메종미네드', '#베드포드일레븐스', '#아노락 재킷', '#이티씨이', '#캔버스/단화', '#컨버스', '#겨울', '#캐주얼', '#스티치', '# 그래픽', '#언밸런스', '# 와이드핏', '#니트/스웨터', '#니틀리', '#셔츠/블라우스

In [ ]:
# 위의 코드를 통해 만들어진 '모든 이미지가 가지고 있는 태그 리스트'에서 <무신사에서 판매하는 제품 정보>와 <코디 스타일> 태그를 제거하는 코드임.
for product in product_list:
  for tag in uniq_tags_list:
      if product in tag: 
          print('>> remove: ' + tag)
          uniq_tags_list.remove(tag)

In [ ]:
print(uniq_tags_list)
print(len(uniq_tags_list))

['#겨울', '#스트릿', '#컬러포인트', '#글로시', '# 카고', '# 와이드핏', '#컬러블록', '# 퀼팅', '#유니크', '#디앤에스알', '#에러스익셉티드', '#유니폼브릿지', '#이티씨이', '#척', '#투큐브', '#프로스펙스', '#피스메이커', '#겨울', '#아메리칸 캐주얼', '# 퀼팅', '# 와이드핏', '#핀턱', '#컬러포인트', '# 아메카지', '#레테르코모', '#밀로', '#선데이후르츠마켓', '#에러스익셉티드', '#이티씨이', '#클래티지', '#겨울', '#스트릿', '#컬러포인트', '#레터링', '# 코듀로이', '# 플라워', '#페이즐리', '#유니크', '#그레고리', '#레테르코모', '#선데이후르츠마켓', '#아웃 오브 트렁크', '#오와이', '#오찌', '#투큐브', '#포콤마', '#겨울', '#캐주얼', '#얼죽코', '# 싱글코트', '#컬러블록', '# 워싱', '# 와이드핏', '# 그래픽', '#얼죽코', '# 싱글코트', '#컬러블록', '# 워싱', '# 와이드핏', '# 그래픽', '#86로드', '#메종미네드', '#베드포드일레븐스', '#이티씨이', '#컨버스', '#겨울', '#캐주얼', '#스티치', '# 그래픽', '#언밸런스', '# 와이드핏', '#니틀리', '#아웃 오브 트렁크', '#톤즈', '#포니', '#겨울', '#캐주얼', '#꾸안꾸', '#레터링', '#컬러블록', '# 코듀로이', '# 카고', '# 스웨이드', '#밀로', '#오찌', '#이벳필드', '#이티씨이', '#피스메이커', '#겨울', '#스트릿', '#레터링', '#컬러포인트', '# 코듀로이', '# 카고', '#유니크', '#밀로', '#바잘', '#선데이후르츠마켓', '#컨버스', '#투큐브', '#겨울', '#댄디', '#뉴트럴톤', '# 싱글코트', '# 레이어드룩', '#파스텔', '#핀턱', '# 와이드핏', '#드로우핏', '#마이크로데이', '#베이스모먼트', '#여밈'

In [ ]:
# 제품 정보와 코디 스타일 태그가 제거된 태그 리스트에서 Unique한 태그의 빈도를 계산하여 빈도 순으로 나열하는 코드임.
count_items = Counter(uniq_tags_list)
print(count_items)

Counter({'#여름': 809, '#캐주얼': 703, '#겨울': 590, '#댄디': 552, '#가을': 523, '#컬러포인트': 465, '#핀턱': 428, '#컬러블록': 411, '#레이어드룩': 398, '#와이드핏': 356, '#그래픽': 318, '#스트릿': 295, '#로고플레이': 293, '#카고': 291, '#뉴트럴톤': 285, '#워싱': 280, '#무신사 스탠다드': 246, '#오버핏': 245, '#스트라이프': 240, '#환절기': 221, '#체크': 219, '#레터링': 218, '#데이트': 216, '#스포티': 194, '#스티치': 182, '#모노톤': 176, '#심볼': 171, '#얼죽코': 168, '#유니크': 165, '#캠퍼스': 161, '#레저스포츠': 151, '#봄': 141, '#미니멀': 135, '#꾸안꾸': 133, '#골프': 124, '#컨버스': 121, '#남친룩': 119, '#비비드': 119, '#톤온톤': 118, '#셋업코디': 107, '#키높이': 106, '#편안한': 106, '#아메리칸 캐주얼': 102, '#레더': 102, '#플리스': 101, '#코듀로이': 100, '#파스텔': 97, '#퀼팅': 89, '#스포츠': 89, '#펀칭': 71, '#힙합': 70, '#반스': 70, '#골프화': 65, '#화사한': 64, '#르마드': 61, '#아디다스': 61, '#닥터마틴': 59, '#시스루': 59, '#아메카지': 58, '#실버태그': 58, '#쿠나썬': 57, '#캠핑': 56, '#출근룩': 56, '#카무플라주': 55, '#에스에스와이': 55, '#쿠어': 54, '#한파': 54, '#레씨토': 54, '#멜란지 마스터': 54, '# 와이드핏': 53, '#골프가방': 53, '#데꼬로소': 53, '#페이즐리': 52, '#그레고리': 52, '#피렌체 아뜨리에': 52, '#뉴발란스': 51, '#원

위의 결과를 바탕으로,

팀 내에서 내부 회의를 통해 2개의 추가적인 카테고리를 적용하기로 함.

자세한 내용은 보고서를 참조